<a href="https://colab.research.google.com/github/rathoddharmendra/kickstarter-platform-analysis/blob/main/kickstarter_platform_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Finding median for the funding-per-project metric

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# from google.colab import auth

# # Authenticate your Google account
# auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

creds = service_account.Credentials.from_service_account_file("/content/storiesmitdee.json")
client = bigquery.Client(credentials=creds, project='data-analytics-ns-470609')

## Working on various KPIs

In [ ]:
query = """
SELECT
  -- APPROX_QUANTILES(funding_per_project, 100)[OFFSET(50)] AS median_funding_per_project
  *
FROM `data-analytics-ns-470609.kickstarter_project_analysis.ks-projects-clean`
"""

df = client.query(query).to_dataframe()
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,status
0,657667332,help me get a 3d printer please!!! (Suspended),3D Printing,Technology,USD,2015-03-29,3500.0,2015-02-27 06:27:40+00:00,0.00,suspended,0,US,0
1,1783647458,Online 3D Printing Marketplace + Local printin...,3D Printing,Technology,EUR,2014-11-03,100000.0,2014-10-04 08:04:23+00:00,0.00,failed,0,NL,0
2,474745489,Image to Life,3D Printing,Technology,USD,2017-07-17,100000.0,2017-06-17 00:44:29+00:00,0.00,failed,0,US,0
3,1853034878,"Modular 3D Printing Kit, a DYI RepRap Anyone C...",3D Printing,Technology,USD,2015-09-16,5000.0,2015-08-17 21:49:29+00:00,0.00,canceled,0,US,0
4,1015381354,Krimson Titan Customs: Project Onyx (Canceled),3D Printing,Technology,USD,2015-09-05,1200.0,2015-08-06 17:32:15+00:00,0.00,canceled,0,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375087,1929840910,Double Fine Adventure,Video Games,Games,USD,2012-03-14,400000.0,2012-02-09 02:52:52+00:00,3336371.92,successful,87142,US,1
375088,1755266685,The Veronica Mars Movie Project,Narrative Film,Film & Video,USD,2013-04-13,2000000.0,2013-03-13 15:42:22+00:00,5702153.38,successful,91585,US,1
375089,557230947,"Bring Reading Rainbow Back for Every Child, Ev...",Web,Technology,USD,2014-07-02,1000000.0,2014-05-28 15:05:45+00:00,5408916.95,successful,105857,US,1
375090,1386523707,Fidget Cube: A Vinyl Desk Toy,Product Design,Design,USD,2016-10-20,15000.0,2016-08-30 22:02:09+00:00,6465690.30,successful,154926,US,1


In [ ]:
df['pledged'].median()

620.0

*median per project* - goal: 5500 vs pledged: 620

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375092 entries, 0 to 375091
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   ID             375092 non-null  Int64              
 1   name           375088 non-null  object             
 2   category       375092 non-null  object             
 3   main_category  375092 non-null  object             
 4   currency       375092 non-null  object             
 5   deadline       375092 non-null  dbdate             
 6   goal           375092 non-null  float64            
 7   launched       375092 non-null  datetime64[us, UTC]
 8   pledged        375092 non-null  float64            
 9   state          375092 non-null  object             
 10  backers        375092 non-null  Int64              
 11  country        375092 non-null  object             
 12  status         375092 non-null  Int64              
dtypes: Int64(3), datetime64[us, U

In [ ]:
df.describe()

,ID,goal,pledged,backers,status
count,375092.0,3.750920e+05,3.750920e+05,375092.0,375092.0
mean,1074635644.384807,4.950586e+04,9.753464e+03,106.622426,0.364591
std,619130372.943478,1.189000e+06,9.608663e+04,911.431981,0.481316
min,5971.0,1.000000e-02,0.000000e+00,0.0,0.0
25%,538080126.0,2.000000e+03,3.100000e+01,2.0,0.0
50%,1075218664.0,5.500000e+03,6.200000e+02,12.0,0.0
75%,1610133769.25,1.650000e+04,4.080000e+03,57.0,1.0
max,2147476221.0,1.000000e+08,2.033899e+07,219382.0,1.0


In [ ]:
## median per projects - for successful projects
df['pledged'][df['status'] == 1].median()

5051.0

In [ ]:
# px.bar(df.pledged)

In [ ]:
## /* choosing median over mean as it is less affected by outliers */
# 5. **Top Subcategories by Money Raised**
# Which niches raise the most.

df.groupby('main_category')['pledged'].median().sort_values(ascending=False)

,pledged
main_category,
Design,1990.0
Dance,1825.0
Theater,1500.0
Comics,1480.0
Games,1282.5
Music,1005.0
Film & Video,736.0
Art,424.0
Technology,331.0


In [ ]:
px.bar(df.groupby('main_category')['pledged'].median().sort_values(ascending=False))

In [ ]:
df.category.value_counts()

,count
category,
Product Design,22312
Documentary,16138
Tabletop Games,14180
Music,13493
Shorts,12357
...,...
Residencies,69
Letterpress,49
Chiptune,35


In [ ]:
df['deadline'] = pd.to_datetime(df['deadline'], errors='raise')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375092 entries, 0 to 375091
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   ID             375092 non-null  Int64              
 1   name           375088 non-null  object             
 2   category       375092 non-null  object             
 3   main_category  375092 non-null  object             
 4   currency       375092 non-null  object             
 5   deadline       375092 non-null  datetime64[ns]     
 6   goal           375092 non-null  float64            
 7   launched       375092 non-null  datetime64[us, UTC]
 8   pledged        375092 non-null  float64            
 9   state          375092 non-null  object             
 10  backers        375092 non-null  Int64              
 11  country        375092 non-null  object             
 12  status         375092 non-null  Int64              
dtypes: Int64(3), datetime64[ns](1

In [ ]:
# 6. **Average Campaign Duration (days)**

df['campaign_duration'] = (df['deadline'] - df['launched']).dt.days

In [ ]:
df['campaign_duration'].value_counts()

,count
campaign_duration,
29,167536
59,33318
44,17643
30,13119
34,10020
...,...
82,54
76,52
73,46


In [ ]:
df.groupby(df['campaign_duration'].value_counts())['status'].sum().sort_values(ascending=True)

,status
count,
22.0,0
39.0,0
46.0,0
52.0,0
54.0,0
...,...
10020.0,0
13119.0,0
17643.0,0


In [ ]:
df.campaign_duration.median()

29.0

In [ ]:
## avg campaign duration for successful and failed projecsts (from status category)

df.groupby('status')['campaign_duration'].median()

,campaign_duration
status,
0,29.0
1,29.0


In [ ]:
df.groupby('main_category')['campaign_duration'].median().sort_values(ascending=False)

,campaign_duration
main_category,
Art,29.0
Comics,29.0
Crafts,29.0
Dance,29.0
Design,29.0
Fashion,29.0
Film & Video,29.0
Food,29.0
Games,29.0


In [ ]:
# 7. **Average Backers per Project**

df['backers'].mean()

np.float64(106.62242596482996)

In [ ]:
df.groupby('main_category')['backers'].median().sort_values(ascending=False)

,backers
main_category,
Comics,40.0
Design,29.0
Games,29.0
Dance,27.0
Theater,24.0
Music,18.0
Film & Video,12.0
Art,10.0
Publishing,7.0


In [ ]:
# 8. **Geographical Distribution**

df.groupby('country')['status'].sum().sort_values(ascending=False)

,status
country,
US,111039
GB,12346
CA,4266
AU,2080
DE,1012
FR,960
NL,652
ES,544
SE,529


In [ ]:
px.bar(df.groupby('country')['status'].sum().sort_values(ascending=False))

In [ ]:
# vs the no of pitches from countries
df.groupby('country')['status'].count().sort_values(ascending=False)

,status
country,
US,292621
GB,33672
CA,14756
AU,7839
DE,4171
FR,2939
IT,2878
NL,2868
ES,2276


In [ ]:
px.bar(df.groupby('country')['status'].count().sort_values(ascending=False))